# Eco Score Pipeline for beautofuel

In [33]:
%load_ext autoreload
%autoreload 2

from influxdb import DataFrameClient

from lib.utils.constants import INFLUXDB_HOST, INFLUXDB_PORT, INFLUXDB_USER, INFLUXDB_PASSWORD, INFLUXDB_DB

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Pipeline configuration

In [34]:
# Database client
grafanadb = DataFrameClient(
    host=INFLUXDB_HOST,
    port=INFLUXDB_PORT,
    database=INFLUXDB_DB,
    username=INFLUXDB_USER,
    password=INFLUXDB_PASSWORD
)

user = "zlatka"

## Helper functions

In [35]:
def get_query_result_value(result, field):
    return result['tracks'][field][0]

def get_query_end(phase):
    return "FROM \"tracks\" WHERE (\"phase\"='{}' AND \"user\"='{}')".format(phase, user)

## Setup Queries

In [36]:
# All tracks query
query = "SELECT \"length\", \"consumption\", \"fuelConsumed\", \"speed\" {};".format(get_query_end(1))

# Consumption queries
stdddev_consumption_query = "SELECT stddev(\"consumption\") {};".format(get_query_end(1))
mean_consumption_query = "SELECT mean(\"consumption\") {};".format(get_query_end(1))
min_consumption_query = "SELECT min(\"consumption\") {};".format(get_query_end(1))
max_consumption_query = "SELECT max(\"consumption\") {};".format(get_query_end(1))

# Speed queries
stdddev_speed_query = "SELECT stddev(\"speed\") {};".format(get_query_end(1))
mean_speed_query = "SELECT mean(\"speed\") {};".format(get_query_end(1))
min_speed_query = "SELECT min(\"speed\") {};".format(get_query_end(1))
max_speed_query = "SELECT max(\"speed\") {};".format(get_query_end(1))

# Fuel consumed queries
stdddev_fuel_consumed_query = "SELECT stddev(\"fuelConsumed\") {};".format(get_query_end(1))
mean_fuel_consumed_query = "SELECT mean(\"fuelConsumed\") {};".format(get_query_end(1))
min_fuel_consumed_query = "SELECT min(\"fuelConsumed\") {};".format(get_query_end(1))
max_fuel_consumed_query = "SELECT max(\"fuelConsumed\") {};".format(get_query_end(1))

# Length queries
stdddev_length_query = "SELECT stddev(\"length\") {};".format(get_query_end(1))
mean_length_query = "SELECT mean(\"length\") {};".format(get_query_end(1))
min_length_query = "SELECT min(\"length\") {};".format(get_query_end(1))
max_length_query = "SELECT max(\"length\") {};".format(get_query_end(1))

## Execute Queries

In [37]:
# Execute all tracks query
tracks = grafanadb.query(query)

# Execute consumption queries
stddev_consumption = get_query_result_value(grafanadb.query(stdddev_consumption_query), 'stddev')
mean_consumption = get_query_result_value(grafanadb.query(mean_consumption_query), 'mean')
min_consumption = get_query_result_value(grafanadb.query(min_consumption_query), 'min')
max_consumption = get_query_result_value(grafanadb.query(max_consumption_query), 'max')

# Execute speed queries
stddev_speed = get_query_result_value(grafanadb.query(stdddev_speed_query), 'stddev')
mean_speed = get_query_result_value(grafanadb.query(mean_speed_query), 'mean')
min_speed = get_query_result_value(grafanadb.query(min_speed_query), 'min')
max_speed = get_query_result_value(grafanadb.query(max_speed_query), 'max')

# Execute fuel consumed queries
stddev_fuel_consumed = get_query_result_value(grafanadb.query(stdddev_fuel_consumed_query), 'stddev')
mean_fuel_consumed = get_query_result_value(grafanadb.query(mean_fuel_consumed_query), 'mean')
min_fuel_consumed = get_query_result_value(grafanadb.query(min_fuel_consumed_query), 'min')
max_fuel_consumed = get_query_result_value(grafanadb.query(max_fuel_consumed_query), 'max')

# Execute length queries
stddev_length = get_query_result_value(grafanadb.query(stdddev_length_query), 'stddev')
mean_length = get_query_result_value(grafanadb.query(mean_length_query), 'mean')
min_length = get_query_result_value(grafanadb.query(min_length_query), 'min')
max_length = get_query_result_value(grafanadb.query(max_length_query), 'max')

In [38]:
# Print all tracks query results
# print(tracks['tracks'])

# Print consumption queries results
print("consumption_100_std:", stddev_consumption)
print("consumption_100_mean:", mean_consumption)
print("consumption_100_min:", min_consumption)
print("consumption_100_max:", max_consumption)

print()

# Print speed queries results
print("speed_stddev:", stddev_speed)
print("speed_mean:", mean_speed)
print("speed_min:", min_speed)
print("speed_max:", max_speed)

print()

# Print fuel consumed queries results
print("fuel_consumed_stddev:", stddev_fuel_consumed)
print("fuel_consumed_mean:", mean_fuel_consumed)
print("fuel_consumed_min:", min_fuel_consumed)
print("fuel_consumed_max:", max_fuel_consumed)

print()

# Print length queries results
print("length_stddev:", stddev_length)
print("length_mean:", mean_length)
print("length_min:", min_length)
print("length_max:", max_length)

consumption_100_std: 1.5725679635551524
consumption_100_mean: 6.581999999999999
consumption_100_min: 4.65
consumption_100_max: 8.51

speed_stddev: 5.8221602520026865
speed_mean: 46.58
speed_min: 37.78
speed_max: 53.33

fuel_consumed_stddev: 0.6951474663695466
fuel_consumed_mean: 1.1960000000000002
fuel_consumed_min: 0.17
fuel_consumed_max: 2.0

length_stddev: 8.98299878657569
length_mean: 18.34294036000009
length_min: 2.382502264852782
length_max: 23.495476139074285


## Eco-score Calculation

In [52]:
tracks_2_query = "SELECT \"length\", \"consumption\", \"fuelConsumed\", \"speed\" {};".format(get_query_end(2))

tracks_2 = grafanadb.query(tracks_2_query)

track_index = 1

for _, track_row in tracks_2['tracks'].iterrows():
    track_2_consumption = track_row['consumption']

    track_2_eco_score = None

    lower_consumption_limit = mean_consumption - stddev_consumption
    upper_consumption_limit = mean_consumption + stddev_consumption

    if track_2_consumption >= upper_consumption_limit:
        eco_score = 0
    elif track_2_consumption <= lower_consumption_limit:
        eco_score = 100
    else:
        track_2_consumption = track_2_consumption - lower_consumption_limit
        upper_consumption_limit = upper_consumption_limit - lower_consumption_limit

        eco_score = int((track_2_consumption / upper_consumption_limit) * 100)

    print("{}. eco_score: {}".format(track_index, eco_score))
    track_index += 1

1. eco_score: 100
2. eco_score: 75
3. eco_score: 9
4. eco_score: 0
5. eco_score: 65
6. eco_score: 74
7. eco_score: 15
8. eco_score: 0
